In [1]:
import openai

RATE_LIMIT = 0.5 # seconds after every OpenAI API call
with open('openai_cred.key', 'r') as inf:
    apiKey = inf.readline().strip()
    orgKey = inf.readline().strip()
    projKey = inf.readline().strip()

client = openai.OpenAI(
      api_key=apiKey,
      organization=orgKey,
      project=projKey
)

def prompt_gpt(prompt, tokens=250):
    completion = client.chat.completions.create(
                    model="gpt-4o",
                    messages=[
                    {"role": "user", "content": prompt}],
                    temperature=0,
                    max_tokens=tokens
    )
    return completion.choices[0].message.content



In [2]:
prompt_gpt("How many r are in strawberry?")

'The word "strawberry" contains three "r" letters.'

# Load the Debate Data

In [2]:
# load the abcnews_transcript file from the data folder
with open('../data/abcnews_transcript.txt', 'r') as file:
    data = file.read()

trumps_responses = []
kamalas_responses = []

trump_identifier = "FORMER PRESIDENT DONALD TRUMP: "
kamala_identifier = "VICE PRESIDENT KAMALA HARRIS: "

for line in data.split('\n'):
    if line.startswith(trump_identifier):
        trumps_responses.append(line[len(trump_identifier):])
    elif line.startswith(kamala_identifier):
        kamalas_responses.append(line[len(kamala_identifier):])

print('Number of Trump responses:', len(trumps_responses))
print('Number of Kamala responses:', len(kamalas_responses))

# print the first response from each
print()
print('Trump:', trumps_responses[0])
print('Kamala:', kamalas_responses[0])

Number of Trump responses: 74
Number of Kamala responses: 34

Trump: Nice to see you. Have fun.
Kamala: Kamala Harris. Let's have a good debate.


In [3]:
prompt_01 = "Given the following response in a debate, does the response \
make use of figurative frames?\n\
If so, say yes and list the figurative \
frames with their explanation in brackets. \
Do not use new lines for your response. If no figurative frames\
were used, say 'no figurative frames'. Response: "

def annotate_response(response, speaker):
    model_answer = prompt_gpt(prompt_01+response)
    return model_answer

In [5]:
annotate_response(trumps_responses[2], 'Trump')

'Yes. Figurative frames: "country buster" (inflation is described as something that can break up countries), "taking over the towns" (immigrants are described as if they are invading and seizing control of towns), "destroying our country" (immigrants are described as causing widespread damage to the nation).'

In [6]:
from tqdm import tqdm

# annotate all responses by trump and write to a file
with open('trump_GPT4_FigFrames.txt', 'w') as file:
    file.write("ID\tResponse\tAnnotation\n")

    for idx, response in tqdm(enumerate(trumps_responses)):
        annotation = annotate_response(response, 'Trump')
        file.write(str(idx)+"\t"+response+"\t"+annotation+'\n')

0it [00:00, ?it/s]

74it [01:50,  1.49s/it]


In [4]:
from tqdm import tqdm

# annotate all responses by trump and write to a file
with open('harris_GPT4_FigFrames.txt', 'w') as file:
    file.write("ID\tResponse\tAnnotation\n")

    for idx, response in tqdm(enumerate(kamalas_responses)):
        annotation = annotate_response(response, 'Harris')
        file.write(str(idx)+"\t"+response+"\t"+annotation+'\n')

34it [00:44,  1.32s/it]


In [16]:
# annotate all responses by trump and write to a file
with open('trump_GPT4_FigFrames.txt', 'r') as file:
    # read all lines
    lines = file.readlines()

all_frames = ""
for line in lines:
    for part in line.split("\t"):
        if part.lower().startswith("yes"):
            if "frames: " in part:
                all_frames += line.split(": ")[1].strip()
            elif "frames. " in part:
                all_frames += line.split(". ")[1].strip()

all_frames =  all_frames.replace(").", "),")
print(all_frames)


"country buster" (inflation is described as something that can break up countries), "taking over the towns" (immigrants are described as if they are invading and seizing control of towns), "destroying our country" (immigrants are described as causing severe harm to the nation),"I'm an open book" (metaphor for being transparent and honest), "We got hit with a pandemic" (metaphor for being severely affected by the pandemic), "We handed them over a country" (metaphor for transferring control or responsibility), "Nobody's ever seen anything like it" (hyperbole to emphasize the uniqueness of the situation), "We did things that nobody thought possible" (hyperbole to emphasize the extraordinary achievements),"sound bite" (a brief, catchy statement designed for media consumption), "bring up our worth, our value as a country" (metaphorically increasing the nation's economic and social standing), "run-Spot-run" (a reference to simplistic, elementary-level content, implying her plan is overly sim

#### The next line is expensive!

In [24]:
prompt_02 = "Your are provided with a list of figurative frames. \
    The format is the following: \"Frame\" (Explanation). \
        Your task is to identify the most prominent themes across the list and \
            group each frame into a theme, but do not repeat the (Explanation). Here is the list: "

answer @03= prompt_gpt(prompt_02+all_frames, tokens=4096)

# write the answer to a file
with open('trump_GPT4_Themes.txt', 'w') as file:
    file.write(answer)
